In [ ]:
include("Utils.jl") ;

# Some basic computing principles and their relationships to potential software failures

The examples I'm going to give are primarily in [Julia](https://julialang.org/), but the basic ideas are true for all languages. How a particular language deals with a failure, however, will be unique.


## Here is a simple question.

What is the difference between `x`, `y`, and `z` in the code below?

```Julia
x = 1
y = 1.0
z = "1.0"
```

We can have Julia tell us what these variables are:

In [ ]:
x= 1; y = 1.0 ; z = "1.5" ;

In [ ]:
println(typesof(1,1.0,"1.0"))

Each of these representation has implications for what can be done with them. Let's start by looking at `Int64` a computer representation of an integer.

What is the result of the following statements?

- $3 + 1$
- $13527 + 1$
- $9223372036854775807 + 1$

In [ ]:
3+1

In [ ]:
13527+1

In [ ]:
9223372036854775807+1

In [ ]:
bignplus1

## Here is a more subtle question

### Can you identify any notable differences between the following two numbers?

```julia

0.1 ;
0.125 ;
```


### Create variables with these values

In [ ]:
Δ₀ = 0.1

In [ ]:
Δ₁ = 0.125

In [ ]:
typeof(Δ₀), typeof(Δ₁)

Remember how we represent numbers in our __positional number representation__.

The number `1893` means

$$
1\times 10^3 + 8\times 10^2 + 9\times 10^1 +3\times10^0
$$

And the number `18.93` means

$$
1\times 10^1 + 8\times 10^0 + 9\times 10^{-1} + 3\times 10^{-2}
$$

These numbers are __base 10__ because each position represents a different power of 10.

But recall from module 1, computers are __base 2__ (binary).

So what do $\Delta_0$ and $\Delta_1$ look like in binary?  Here is the mathematical representation implemented in floating point numbers:

![IEEE FPN](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/IEEE_754_Double_Floating_Point_Format.svg/1024px-IEEE_754_Double_Floating_Point_Format.svg.png)

$$
\left(-1\right)^{sign}\left(1+\sum_{i=1}^{52}b_{52-i}2^{-1}\right)\times 2^{e-1023}
$$

(There are some details we are going to skip over here. See the Wikipedia article on the [IEEE Floating Point standard](https://en.wikipedia.org/wiki/Double-precision_floating-point_format) for more details.)

the function `floatbits` takes a floating point number and renders it as the bits used in the computer to represent it, inserting an underscore between the sign bit, exponent, and fraction.

In [ ]:
floatbits(Δ₀)

In [ ]:
floatbits(Δ₁)

## Anything notable?

## Here are some other examples

- $\pi$: irrational
- $\frac{1}{3}$ rational (repeating decimal)

In [ ]:
floatbits(Float64(π))

In [ ]:
floatbits(1/3)

## An easy problem

In [ ]:
d = 0.0
while d != 0.3
    d += 0.1
end

In [ ]:
d

In [ ]:
d = 0.0
while d != 1.0
    d += 0.125
end

In [ ]:
d

## Let's implement some functions to compute elapsed time (`etime`)

- `etime64` represents floating point numbers with 64 bits
- `etime32` represents floating point numbers with 32 bits
  - Rounding behavior is mansged by the IEEE floating point standard
- `etimetr` represents floating point numbers with 64 bits
    - We truncate so that we only have 24 bits of the fraction (instead of 52 bits)

In [ ]:
function etime64(et::Int64, ni::Int64)::Float64
    δt = 1.0::Float64/ni
    t = 0.0
    for i in 1:et*ni
        t = t + δt
    end
    t
end

In [ ]:
function etime32(et::Int64, ni::Int64)::Float32
    δt = 1.0f0/ni
    t = 0.0f0
    for i in 1:et*ni
        t = t + δt
    end
    t
end

In [ ]:
function etimetr(et::Int64, ni::Int64)
    δt = 1.0f0/ni
    t = 0.0
    for i in 1:et*ni
        t = mask_float_bitstring(t + δt, 24)
    end
    t
end

### 64 bit

In [ ]:
etime64(3600, 10)

#### What is our error?

In [ ]:
etime64(3600, 10) - 3600

### 32 bit

In [ ]:
etime32(3600, 10)

In [ ]:
etime32(3600, 10) - 3600

### truncated

### This one is slow

In [ ]:
etimetr(3600, 10)

In [ ]:
etimetr(3600, 10) - 3600

### What if we repeat this with a base-2-informed increment?

In [ ]:
etime64(3600, 16), etime64(3600, 16) - 3600

In [ ]:
etime32(3600, 16), etime32(3600, 16) - 3600

In [ ]:
etimetr(3600, 16), etimetr(3600, 16) - 3600